In [2]:
import numpy as np
import pyvista as pv
import pandas as pd
import scipy.io

In [3]:
def nodesExtractor(name): #extracts nodes and their corresponding information
    file_path = 'Networks/Network_Vessels_' + name +'.mat'
    matlab_data = scipy.io.loadmat(file_path)
    # Extract the 'connectivity' field from the 'Data' structured array
    data_structure = matlab_data['nodesC2']
    # Reshape or ensure it's a proper 2D array (if required)
    nodes_data = data_structure.squeeze()
    # Create a DataFrame from the connectivity data
    nodes_df = pd.DataFrame(nodes_data, columns=['NodeID', 'X', 'Y', 'Z', 'Degree'])
    # Save the DataFrame to inspect it
    return nodes_df

def term_nodes_loc(name,rootID):
    nodes = nodesExtractor(name)
    deg1 = nodes[(nodes['Degree'] == 1)]
    termNodes = deg1[deg1['NodeID']!=rootID]
    return termNodes[['X','Y','Z']]

In [4]:
terms = term_nodes_loc('m3p4_060407',369).astype(float)
term_loc = terms.to_numpy()

In [5]:
cloud = pv.PolyData(term_loc)
cloud.plot()

Widget(value='<iframe src="http://localhost:49352/index.html?ui=P_0x150fb8850_0&reconnect=auto" class="pyvista…

In [9]:
volume = cloud.delaunay_3d(alpha=100.)
shell = volume.extract_geometry()
shell.plot()

Widget(value='<iframe src="http://localhost:49352/index.html?ui=P_0x176fc1880_4&reconnect=auto" class="pyvista…